In [1]:
import random

import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras.layers import Dense, Permute, Softmax, Activation, Add

In [2]:
random.seed(42)

print(tf.__version__)
tf.random.set_seed(42)

2.6.0


- tf.keras.layers.Dense

- tf.keras.layers.Permute

In [3]:
ele01 = [random.random() for _ in range(4)]
ele02 = [ele01 for _ in range(5)]
ele03 = [ele02 for _ in range(7)]

X = tf.constant(ele03, dtype=tf.float32)

# print(X)
print(X.shape)
print()

X_tr = Permute((2, 1))(X)
# print(X_tr)
print(X_tr.shape)

(7, 5, 4)

(7, 4, 5)


In [4]:
hidden_state = tf.constant([[random.random() for _ in range(3)] for _ in range(7)], dtype=tf.float32)
cell_state = tf.constant([[random.random() for _ in range(3)] for _ in range(7)], dtype=tf.float32)

print(tf.concat([hidden_state, cell_state], axis=-1).shape)

(7, 6)


In [5]:
n = X.shape[2]

hs = K.repeat(tf.concat([hidden_state, cell_state], axis=-1), n)
print(hs.shape)

(7, 4, 6)


In [6]:
T = 6

print(Dense(T)(hs).shape)

(7, 4, 6)


In [7]:
ux = Dense(T)(Permute((2, 1))(X))
print(ux.shape)

(7, 4, 6)


In [8]:
print(hs[0, 0, :])

print(ux[0, 0, :])

tf.Tensor([0.73647124 0.67669946 0.89217955 0.33659455 0.09274584 0.09671637], shape=(6,), dtype=float32)
tf.Tensor([0.58065337 0.72401965 0.23058392 0.11091197 0.22722673 1.0361034 ], shape=(6,), dtype=float32)


### tf.keras.layers.Add

- tf.math.tanh

- tf.keras.layers.Activation

In [9]:
temp_add = Add()([hs, ux])

print(temp_add.shape)

print(temp_add[0, 0, :])

(7, 4, 6)
tf.Tensor([1.3171246  1.4007192  1.1227635  0.44750652 0.31997257 1.1328198 ], shape=(6,), dtype=float32)


In [10]:
tanh_math_add = tf.math.tanh(temp_add)
print(tanh_math_add.shape)

(7, 4, 6)


In [11]:
tanh_act_add = Activation(activation='tanh')(temp_add)
print(tanh_act_add.shape)

(7, 4, 6)


In [12]:
diff_tanh_add = tanh_math_add - tanh_act_add
print(sum(sum(sum(diff_tanh_add))))

tf.Tensor(0.0, shape=(), dtype=float32)


In [13]:
e_add = Dense(1)(tanh_act_add)
print(e_add[:, :, 0])
print(e_add.shape)

tf.Tensor(
[[-0.4243279  -0.33395013 -0.3741942  -0.36571822]
 [-0.1903434  -0.23177858 -0.20842505 -0.21312569]
 [-0.13270655 -0.15531427 -0.13563734 -0.13891803]
 [-0.4147418  -0.46678478 -0.44110227 -0.44636545]
 [-0.5664377  -0.6363906  -0.6064062  -0.61289597]
 [-0.5827408  -0.6040077  -0.602988   -0.6042776 ]
 [-0.5466778  -0.6104355  -0.5848892  -0.5904835 ]], shape=(7, 4), dtype=float32)
(7, 4, 1)


In [14]:
attn_add = Softmax()(e_add)
print(attn_add[:, :, 0])
print(attn_add.shape)

tf.Tensor(
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]], shape=(7, 4), dtype=float32)
(7, 4, 1)


In [15]:
attn_add = Softmax()(Permute((2, 1))(e_add))
print(attn_add[:, 0, :])
print(attn_add.shape)

tf.Tensor(
[[0.23773518 0.26022202 0.24995755 0.25208518]
 [0.25516924 0.24481231 0.25059682 0.24942161]
 [0.25198263 0.2463498  0.2512452  0.2504223 ]
 [0.25692785 0.24389854 0.2502436  0.24892999]
 [0.2598846  0.2423262  0.24970224 0.24808699]
 [0.25396132 0.24861737 0.24887103 0.24855028]
 [0.25920957 0.24319881 0.24949168 0.24809985]], shape=(7, 4), dtype=float32)
(7, 1, 4)


### tf.concat

- tf.math.tanh

- tf.keras.layers.Activation

In [16]:
temp_concat = tf.concat([hs, ux], axis=-1)

print(temp_concat.shape)

print(temp_concat[0, 0, :])

(7, 4, 12)
tf.Tensor(
[0.73647124 0.67669946 0.89217955 0.33659455 0.09274584 0.09671637
 0.58065337 0.72401965 0.23058392 0.11091197 0.22722673 1.0361034 ], shape=(12,), dtype=float32)


In [17]:
tanh_math_concat = tf.math.tanh(temp_concat)
print(tanh_math_concat.shape)

(7, 4, 12)


In [18]:
tanh_act_concat = Activation(activation='tanh')(temp_concat)
print(tanh_act_concat.shape)

(7, 4, 12)


In [19]:
diff_tanh = tanh_math_concat - tanh_act_concat
print(sum(sum(sum(diff_tanh))))

tf.Tensor(0.0, shape=(), dtype=float32)


In [20]:
e_act = Dense(1)(tanh_act_concat)
print(e_act[:, :, 0])
print(e_act.shape)

tf.Tensor(
[[ 0.1815817   0.08705606  0.11656632  0.10954158]
 [-0.22645688 -0.32098252 -0.29147226 -0.298497  ]
 [-0.18919656 -0.28372222 -0.25421193 -0.2612367 ]
 [-0.1520116  -0.24653722 -0.21702695 -0.2240517 ]
 [-0.17199141 -0.26651704 -0.23700678 -0.24403152]
 [-0.06787258 -0.16239825 -0.13288797 -0.13991272]
 [ 0.17506585  0.08054021  0.11005048  0.10302573]], shape=(7, 4), dtype=float32)
(7, 4, 1)


In [21]:
attn_input_act = Softmax()(e_act)
print(attn_input_act[:, :, 0])
print(attn_input_act.shape)

tf.Tensor(
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]], shape=(7, 4), dtype=float32)
(7, 4, 1)


In [22]:
attn_input_act = Softmax()(Permute((2, 1))(e_act))
print(attn_input_act[:, 0, :])
print(attn_input_act.shape)

tf.Tensor(
[[0.2647358  0.24085778 0.24807149 0.24633494]
 [0.2647358  0.24085778 0.24807149 0.24633496]
 [0.2647358  0.24085777 0.24807149 0.24633494]
 [0.2647358  0.24085778 0.24807149 0.24633496]
 [0.2647358  0.24085778 0.24807149 0.24633496]
 [0.2647358  0.24085777 0.24807149 0.24633494]
 [0.2647358  0.24085778 0.24807149 0.24633496]], shape=(7, 4), dtype=float32)
(7, 1, 4)
